In [1]:
import requests
import json
import os
from dotenv import load_dotenv
import pandas as pd
import folium
import openrouteservice as ors
from geopy.distance import great_circle

In [2]:
def get_token():
    load_dotenv('./.env')
    email = os.environ.get("email")
    password = os.environ.get("password")
    client_id = os.environ.get("X-ClientId")
    pass_key = os.environ.get("passKey")
    url = "https://openapi.emtmadrid.es/v3/mobilitylabs/user/login/"
    headers = {"email": email, "password" : password}
    response = requests.get(url, headers=headers)
    return response.content

In [3]:
get_token()

b'{"code": "01", "description": "Token extend  into control-cache Data recovered  OK, (lapsed: 1540 millsecs)", "datetime": "2023-11-28T15:09:43.093961", "data": [{"XClientId": "09141f1a-bf02-4e23-80ee-e7fa7ea819d2", "nameApp": "OPENAPI MobilityLabs", "updatedAt": "2023-11-13T18:28:08.8400000", "userName": "silviluliuma", "lastUpdate": {"$date": 1701180544982}, "idUser": "26ddfea8-f2ce-438f-89e4-5110c9abd07e", "priv": "U", "tokenSecExpiration": 86399, "email": "valeromsilvia@gmail.com", "tokenDteExpiration": {"$date": 1701270543982}, "flagAdvise": true, "accessToken": "0faf57af-9af2-4629-a3bc-3893469bab0b", "apiCounter": {"current": 10, "dailyUse": 20000, "owner": 0, "licenceUse": "Please mention EMT Madrid MobilityLabs as data source. Thank you and enjoy!", "aboutUses": "If you need to extend the daily use of this API, please, register your App in Mobilitylabs and use your own X-ClientId and  passKey instead of generic login (more info in https://mobilitylabs.emtmadrid.es/doc/new-app 

In [4]:
def get_stations():
    load_dotenv('./.env')
    token = os.environ.get("access_token")
    print(token)
    url = "https://openapi.emtmadrid.es/v3/transport/bicimad/stations/"
    headers = {"accessToken" : token}
    response = requests.get(url, headers = headers).json()
    return response

In [5]:
get_stations()

0faf57af-9af2-4629-a3bc-3893469bab0b


{'code': '00',
 'description': '603 bases recovered',
 'datetime': '2023-11-28T15:09:44.885543',
 'data': [{'activate': 1,
   'address': 'Avenida del Ensanche de Vallecas, 9,',
   'dock_bikes': 25,
   'free_bases': 2,
   'geofence': {'type': 'Polygon',
    'coordinates': [[[-3.613119408, 40.370889958],
      [-3.613119408, 40.369989402],
      [-3.611941152, 40.369989402],
      [-3.611941152, 40.370889958],
      [-3.613119408, 40.370889958]]]},
   'geometry': {'type': 'Point', 'coordinates': [-3.61253028, 40.37043968]},
   'id': 2190,
   'integrator': '3c34ea72-2a03-4a3c-be97-cf32be4b0ebe',
   'light': 1,
   'name': '453 - Avenida del Ensanche de Vallecas, 9',
   'no_available': 0,
   'number': '453',
   'reservations_count': 0,
   'tipo_estacionPBSC': 'FIXED',
   'total_bases': 27,
   'virtualDelete': False,
   'virtual_bikes': [],
   'virtual_bikes_num': 0,
   'code_district': '18',
   'code_suburb': '183',
   'geofenced_capacity': 20,
   'bikesGo': []},
  {'activate': 1,
   'addre

In [6]:
stations = get_stations()
stations_real_time = pd.DataFrame(stations["data"])
stations_real_time[["longitude", "latitude"]] = stations_real_time["geometry"].apply(lambda x: pd.Series(x["coordinates"]))
stations_real_time = stations_real_time.drop(["geofence", "activate", "geometry", "integrator", "reservations_count", "no_available", "tipo_estacionPBSC", "virtualDelete", "virtual_bikes", "virtual_bikes_num", "code_suburb", "geofenced_capacity", "bikesGo"], axis=1)
stations_real_time['coordinates'] = list(zip(stations_real_time['longitude'], stations_real_time['latitude']))
stations_real_time


0faf57af-9af2-4629-a3bc-3893469bab0b


,address,dock_bikes,free_bases,id,light,name,number,total_bases,code_district,longitude,latitude,coordinates
0,"Avenida del Ensanche de Vallecas, 9,",25,2,2190,1,"453 - Avenida del Ensanche de Vallecas, 9",453,27,18,-3.612530,40.370440,"(-3.61253028, 40.37043968)"
1,"Paseo de la Chopera,33,Comunidad de Madrid España",21,2,2205,1,"267 - Paseo de la Chopera, 33",267,23,02,-3.700400,40.395000,"(-3.7003996647238746, 40.39500000000003)"
2,"Paseo de la Castellana nº 122,",19,4,2224,1,150 - Castellana frente a Hermanos Pinzón,150,23,05,-3.690800,40.449100,"(-3.6908, 40.4491)"
3,"Paseo de la Castellana nº 164,Comunidad de Mad...",3,20,2225,0,157- Castellana 164,157,23,05,-3.689415,40.459137,"(-3.6894151, 40.4591366)"
4,"Guetaria , 84b,",23,1,2226,1,395b - 395 - Guetaria 84b,395b,24,12,-3.715691,40.369168,"(-3.71569102, 40.36916772)"
...,...,...,...,...,...,...,...,...,...,...,...,...
598,"Calle Alcalá, 546,",17,10,1982,2,"498 - Calle Alcala, 546",498,27,20,-3.617832,40.443133,"(-3.6178317, 40.44313349999999)"
599,"Calle Santander esquina con Avenida de Filipinas,",11,16,1608,2,202 - Parque Santander,202,27,07,-3.708300,40.441300,"(-3.7083, 40.4413)"
600,"565- Playa de Riazor, 66,",3,21,2260,0,"565 - Playa de Riazor, 66",565,24,21,-3.583887,40.475636,"(-3.5838869, 40.47563573)"
601,"Avenida del Doctor Arce nº 45,",0,23,1553,0,147 - Doctor Arce 45,147,23,05,-3.679400,40.448400,"(-3.6794, 40.4484)"


In [7]:
def get_district(df, district_number):
    result = df[df["code_district"] == district_number]
    return result

district_03 = get_district(stations_real_time, "03")
district_03

,address,dock_bikes,free_bases,id,light,name,number,total_bases,code_district,longitude,latitude,coordinates
67,"Calle O'Donnell nº 28,",2,25,1465,0,61 - Narváez,61,27,03,-3.675314,40.421455,"(-3.6753139, 40.4214551)"
68,"Calle Ibiza nº 62,",2,21,1467,0,63 - Ibiza,63,23,03,-3.670896,40.417924,"(-3.6708959, 40.4179237)"
69,"Calle Antonio Maura nº 15,",18,6,1468,1,65 - Antonio Maura,65,24,03,-3.689425,40.416701,"(-3.689424797942769, 40.4167014469994)"
71,"Calle Espalter nº 1,",7,12,1470,2,68 - Espalter,68,19,03,-3.691323,40.412769,"(-3.6913232, 40.4127692)"
72,"Avenida de Alfonso XII nº 54,",10,17,1471,2,69 - Puerta del Ángel Caído,69,27,03,-3.688822,40.409808,"(-3.688822, 40.409808)"
73,"Avenida de Menéndez Pelayo nº 63,",13,14,1472,2,70 - Puerta del Doce de Octubre,70,27,03,-3.677873,40.415382,"(-3.677872542327878, 40.415382385574716)"
74,"Calle Doce de Octubre nº 28,",6,17,1473,0,71 - Doce de Octubre,71,23,03,-3.673900,40.416100,"(-3.6739, 40.4161)"
75,"Calle Doctor Esquerdo nº 99,",6,17,1474,0,72 - Sainz de Baranda,72,23,03,-3.669600,40.415900,"(-3.6696, 40.4159)"
76,"Avenida de Nazaret nº 7,",5,18,1475,0,73 - Plaza de los Astros,73,23,03,-3.668700,40.411500,"(-3.6687, 40.4115)"
77,"Avenida de Menéndez Pelayo nº 69,",11,16,1476,2,74 - Puerta del Pacífico,74,27,03,-3.676589,40.412063,"(-3.676588763788984, 40.41206290978386)"


In [8]:
district_01 = stations_real_time[stations_real_time["code_district"]== "01"]
district_02 = stations_real_time[stations_real_time["code_district"]== "02"]
district_03 = stations_real_time[stations_real_time["code_district"]== "03"]
district_04 = stations_real_time[stations_real_time["code_district"]== "04"]
district_05 = stations_real_time[stations_real_time["code_district"]== "05"]
district_06 = stations_real_time[stations_real_time["code_district"]== "06"]
district_07 = stations_real_time[stations_real_time["code_district"]== "07"]
district_08 = stations_real_time[stations_real_time["code_district"]== "08"]
district_09 = stations_real_time[stations_real_time["code_district"]== "09"]
district_10 = stations_real_time[stations_real_time["code_district"]== "10"]
district_11 = stations_real_time[stations_real_time["code_district"]== "11"]
district_12 = stations_real_time[stations_real_time["code_district"]== "12"]
district_13 = stations_real_time[stations_real_time["code_district"]== "13"]
district_14 = stations_real_time[stations_real_time["code_district"]== "14"]
district_15 = stations_real_time[stations_real_time["code_district"]== "15"]
district_16 = stations_real_time[stations_real_time["code_district"]== "16"]
district_17 = stations_real_time[stations_real_time["code_district"]== "17"]
district_18 = stations_real_time[stations_real_time["code_district"]== "18"]
district_19 = stations_real_time[stations_real_time["code_district"]== "19"]
district_20 = stations_real_time[stations_real_time["code_district"]== "20"]
district_21 = stations_real_time[stations_real_time["code_district"]== "21"]

In [9]:
district_02

,address,dock_bikes,free_bases,id,light,name,number,total_bases,code_district,longitude,latitude,coordinates
1,"Paseo de la Chopera,33,Comunidad de Madrid España",21,2,2205,1,"267 - Paseo de la Chopera, 33",267,23,02,-3.700400,40.395000,"(-3.7003996647238746, 40.39500000000003)"
62,"Calle Ribera de Curtidores nº 28,",17,10,1450,2,46 - Ribera de Curtidores,46,27,02,-3.707126,40.405315,"(-3.7071259, 40.4053153)"
109,Calle Teresa López Valcárcel esq. Teniente Cor...,24,0,2071,1,270 - Calle Teresa López Valcárcel,270,24,02,-3.693288,40.388629,"(-3.69328771, 40.38862908)"
152,"Calle Delicias, 45,",14,5,2117,1,"273 - Calle Delicias, 45",273,19,02,-3.690918,40.403819,"(-3.69091831, 40.40381914)"
202,"Glorieta de Embajadores nº 6,",3,21,1451,0,47 - Embajadores 1,47,24,02,-3.702800,40.404577,"(-3.7027998, 40.4045772)"
226,"PASEO DE LOS MELANCÓLICOS 73,",9,14,2247,2,280 - PASEO DE LOS MELANCÓLICOS 73,280,23,02,-3.718569,40.401782,"(-3.718569, 40.401782)"
239,"271 - Calle Amaltea , 1,",23,1,2265,1,"271 - Calle Amaltea , 1",271,24,02,-3.683170,40.394706,"(-3.68317, 40.394706)"
241,"Paseo de Juan Antonio Vallejo Nájera Botas, 28,",13,9,2267,2,"275 - Juan Antonio Vallejo Nájera Botas, 25",275,22,02,-3.708212,40.402054,"(-3.7082123, 40.4020545)"
243,"Calle Méndez Alvaro, 44,",23,0,2269,1,"274 - Méndez Alvaro, 73",274,23,02,-3.681343,40.396758,"(-3.68134346, 40.39675755)"
313,"Paseo de los Pontones, 45,",5,18,2299,0,"278 - Paseo de los Pontones, 45",278,23,02,-3.719600,40.404043,"(-3.7196, 40.4040433)"


In [10]:
def get_light1(df):
    df_light1 = df[df["light"]==1]
    return df_light1

In [11]:
get_light1(district_02)

,address,dock_bikes,free_bases,id,light,name,number,total_bases,code_district,longitude,latitude,coordinates
1,"Paseo de la Chopera,33,Comunidad de Madrid España",21,2,2205,1,"267 - Paseo de la Chopera, 33",267,23,02,-3.700400,40.395000,"(-3.7003996647238746, 40.39500000000003)"
109,Calle Teresa López Valcárcel esq. Teniente Cor...,24,0,2071,1,270 - Calle Teresa López Valcárcel,270,24,02,-3.693288,40.388629,"(-3.69328771, 40.38862908)"
152,"Calle Delicias, 45,",14,5,2117,1,"273 - Calle Delicias, 45",273,19,02,-3.690918,40.403819,"(-3.69091831, 40.40381914)"
239,"271 - Calle Amaltea , 1,",23,1,2265,1,"271 - Calle Amaltea , 1",271,24,02,-3.683170,40.394706,"(-3.68317, 40.394706)"
243,"Calle Méndez Alvaro, 44,",23,0,2269,1,"274 - Méndez Alvaro, 73",274,23,02,-3.681343,40.396758,"(-3.68134346, 40.39675755)"
320,"Paseo virgen de puerto, 25,",14,1,2304,1,"277 - Paseo Virgen del Puerto, 25",277,15,02,-3.721178,40.411622,"(-3.721178, 40.411622)"
343,"279 - Paseo de los Melancólicos, 1,",22,1,2331,1,"279 - Paseo de los Melancólicos, 1",279,23,02,-3.718482,40.410445,"(-3.7184816, 40.4104448)"
345,"Paseo de las Yeserías, 15,",18,5,2333,1,"276 - Paseo de las Yeserías, 15",276,23,02,-3.709917,40.399005,"(-3.7099166, 40.39900539)"
362,"Calle del tejo,",22,1,2305,1,272 - Calle del tejo,272,23,02,-3.679186,40.396630,"(-3.679186, 40.39663)"
396,"Calle Juan Martín el Empecinado nº 16,",23,0,1524,1,118 - Juan Martín,118,23,02,-3.688241,40.400781,"(-3.6882407, 40.400781)"


In [12]:
def get_light0(df):
    df_light0 = df[df["light"]==0]
    return df_light0

In [13]:
get_light0(district_02)

,address,dock_bikes,free_bases,id,light,name,number,total_bases,code_district,longitude,latitude,coordinates
202,"Glorieta de Embajadores nº 6,",3,21,1451,0,47 - Embajadores 1,47,24,02,-3.702800,40.404577,"(-3.7027998, 40.4045772)"
313,"Paseo de los Pontones, 45,",5,18,2299,0,"278 - Paseo de los Pontones, 45",278,23,02,-3.719600,40.404043,"(-3.7196, 40.4040433)"
515,"Paseo Santa María de la Cabeza nº 29,",5,18,1574,0,168 - Batalla del Salado,168,23,02,-3.695900,40.404600,"(-3.6959, 40.4046)"
516,"Calle Toledo nº 181,",7,17,1575,0,169 - Pirámides,169,24,02,-3.713823,40.401589,"(-3.713823, 40.401589)"
518,"Calle Embajadores nº 97,",1,26,1577,0,171 - Embajadores-Cáceres,171,27,02,-3.698402,40.399701,"(-3.6984023, 40.3997013)"
521,"Paseo de Yeserías nº 45,",8,19,1580,0,174 - Madrid Río-Yeserías,174,27,02,-3.706526,40.397430,"(-3.7065258846557603, 40.39742958306358)"


In [14]:
def find_nearest_to_coords(df, coords):
    station_coordinates = df['coordinates'].tolist()
    nearest_station = min(station_coordinates, key=lambda coord: great_circle(coord, vehicle_start).meters)
    return nearest_station

In [15]:
load_dotenv('../.env')
client = ors.Client(key=os.environ.get("openroute_api_key"))

# Function to find the nearest station to given coordinates
def find_nearest_to_coords(df, coords):
    station_coordinates = df['coordinates'].tolist()
    nearest_station = min(station_coordinates, key=lambda coord: great_circle(coord, coords).meters)
    return nearest_station

# Function to create a route between two sets of coordinates
def create_route(client, start_coords, end_coords):
    return client.directions(
        coordinates=[start_coords, end_coords],
        profile='driving-car',
        format='geojson'
    )

# Coordenadas de inicio (supongo que empiezan la ruta en la central de la EMT)
vehicle_start = [-3.6823731969472644, 40.46209827032537]

# Creo un mapa con Folium
m = folium.Map(location=[vehicle_start[1], vehicle_start[0]], zoom_start=12)

# Añado marcador morado para la central (vehicle_start)
folium.Marker(location=[vehicle_start[1], vehicle_start[0]], popup='CENTRAL EMT', icon=folium.Icon(color='purple')).add_to(m)

distrito02_low = get_light0(district_02).copy()
distrito02_high = get_light1(district_02).copy()

distrito02_high['visited'] = False
distrito02_low['visited'] = False

# Inicio de la ruta
current_coords = vehicle_start
van = "empty"

coords_list = [current_coords]

stop_counter = 1

for i in range(100):
    if van == "empty":
        current_coords = coords_list[-1]
        # Verifica si quedan estaciones sin visitar
        if not distrito02_high.loc[~distrito02_high['visited'] & (distrito02_high['light'] == 1)].empty:
            nearest_station = find_nearest_to_coords(distrito02_high.loc[~distrito02_high['visited'] & (distrito02_high['light'] == 1)], current_coords)
            coords_list.append(nearest_station)
            # Actualiza 'visited' y 'light' en el DataFrame original
            distrito02_high.loc[distrito02_high['coordinates'] == nearest_station, 'visited'] = True
            distrito02_high.loc[distrito02_high['coordinates'] == nearest_station, 'light'] = 2
            route = create_route(client, coords_list[-2], coords_list[-1])
            van = "full"
            folium.Marker(location=[nearest_station[1], nearest_station[0]],
                          popup=f"Station with high occupation\nNumber: {stop_counter}",
                          icon=folium.Icon(color='orange')).add_to(m)
            stop_counter += 1
            folium.PolyLine(locations=[coord[::-1] for coord in route['features'][0]['geometry']['coordinates']],
                            color='red').add_to(m)
    elif van == "full":
        current_coords = coords_list[-1]
        # Verifica si quedan estaciones sin visitar
        if not distrito02_low.loc[~distrito02_low['visited'] & (distrito02_low['light'] == 0)].empty:
            nearest_station = find_nearest_to_coords(distrito02_low.loc[~distrito02_low['visited'] & (distrito02_low['light'] == 0)], current_coords)
            coords_list.append(nearest_station)
            # Actualiza 'visited' y 'light' en el DataFrame original
            distrito02_low.loc[distrito02_low['coordinates'] == nearest_station, 'visited'] = True
            distrito02_low.loc[distrito02_low['coordinates'] == nearest_station, 'light'] = 2
            route = create_route(client, coords_list[-2], coords_list[-1])
            van = "empty"
            folium.Marker(location=[nearest_station[1], nearest_station[0]],
                          popup=f"Station with low occupation\nNumber: {stop_counter}",
                          icon=folium.Icon(color='green')).add_to(m)
            stop_counter += 1
            folium.PolyLine(locations=[coord[::-1] for coord in route['features'][0]['geometry']['coordinates']],
                            color='red').add_to(m)
final_route = create_route(client, coords_list[-1], vehicle_start)
folium.PolyLine(locations=[coord[::-1] for coord in final_route['features'][0]['geometry']['coordinates']],
                            color='red').add_to(m)

m